In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.dates as mpl_dates
import requests
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint

import streamlit as st
import sys
import subprocess

import os
import joblib
from datetime import datetime
import time
import mplfinance as mpf
import warnings
warnings.filterwarnings('ignore')



# define variables
coins = "BTC,ETH,ADA,BNB,XRP,LTC,BCH,DOGE,XLM,AAVE,DASH,NEXO,ETC,XMR,WAVES,MIOTA,ICX,DOT,EOS,SOL"
coin_list = coins.split(",")  # ['BTC', ' ETH'...]
coin_dictionary = {coin: coin_list.index(coin) for coin in coin_list}

# load model, encoder, and scaler
scaler_ = joblib.load('myscaler.joblib')
label_encoder_ = joblib.load('mylabel_encoder.joblib')
model_ = joblib.load('mymodel.joblib')

# define functions
def getcoindata(coin_name_, interval_, start_date=None):
    end_date = datetime.datetime.today().strftime('%Y-%m-%d')

    if start_date is None:
        start_date = (datetime.datetime.today().date() - pd.DateOffset(years=1)).date().strftime('%Y-%m-%d')

    crypto2 = yf.Ticker(f"{coin_name_}-USD").history(start=start_date, end=end_date, interval=interval_)

    # Add the "crypto_name" column if it's not present
    if "crypto_name" not in crypto2.columns:
        crypto2["crypto_name"] = coin_name_

    return crypto2

def myScaler_prod(to_scale):
    global scaler_
    data_ = data.copy()
    data_scaled_ = scaler_.transform(data_)
    data_scaled_ = pd.DataFrame(data_scaled_, columns=data_.columns)
    data_scaled_.index = data_.index
    return data_scaled_

def myInverseScale_prod(data):
    global scaler_
    data_scaler_inverse = scaler_.inverse_transform(data)
    data_scaler_inverse = pd.DataFrame(data_scaler_inverse, columns=data.columns)
    data_scaler_inverse.index = data.index
    return data_scaler_inverse

def execute_program(coin="ETH", time_string="1d"):
    global model_

    # Fetch data
    start_date = '2018-01-01'
    if time_string.endswith('d'):
        num_days = int(time_string[:-1])
        start_date = str(datetime.today().date() - pd.DateOffset(days=num_days))

    df_test = getcoindata(coin_name_=coin, interval_=time_string, start_date=start_date)

    # Label encoding
    df_test["crypto_encoded"] = label_encoder_.transform(df_test["crypto_name"])

    # Scaling
    to_scale_ = df_test[["Open", "High", "Low", "Close", "crypto_encoded"]]
    df_scaled_ = myScaler_prod(to_scale_)

    # Prediction
    predicted_ = model_.predict(df_scaled_[["Open", "High", "Low", "crypto_encoded"]])

    # Inverse scaling
    inverse_predicted = df_scaled_.copy()
    inverse_predicted["Close"] = predicted_
    inverse_predicted = myInverseScale_prod(inverse_predicted[["Open", "High", "Low", "Close", "crypto_encoded"]])

    # Merge
    df_test["predicted"] = inverse_predicted["Close"]

    # Return dataset
    return df_test

def calculate_profit_price(starting_price, target_price):
    profit = target_price - starting_price
    return round(profit, 2)

def calculate_confidence_level(predicted_value, actual_value):
    if predicted_value > actual_value:
        confidence = (predicted_value - actual_value) / actual_value
    else:
        confidence = (actual_value - predicted_value) / actual_value
    return round(1 - confidence, 2)

def display_top_stories():
    stories = [
        {
            "title": "Top 4 new cryptocurrencies 2023",
            "description": "New Cryptocurrencies 2023",
            "source": "CoinsPaid Media",
            "published_at": "Updates every hour",
            "url": "https://coinspaidmedia.com/business/top-4-new-cryptocurrencies-2023/"
        },
        {
            "title": "Cryptocurrency News",
            "description": "Latest Crypto and Bitcoin News",
            "source": "Yahoo finance",
            "published_at": "Updates every hour",
            "url": "https://finance.yahoo.com/topic/crypto/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAAM2TswRjGJmFYQ0QHIb7oLhP_5RDjIAkGGf76b2bj-tn1nCR6XLhO2RB3Olx25ftGcoOiMTJtLZeAU5QEoAiaWhmJEmDIn5Fa8iNX9cCkd-98M-CWIsZw8AWRIBBv72f4YQQwaGEmiVlYjevU7-ueW2hvJIrHCFAayhkJI3lterf"
        },
        {
            "title": "Cryptocurrency News",
            "description": "Insights into the biggest event shaping crypto industry",
            "source": "Coin Market",
            "published_at": "Updates every hour",
            "url": "https://coinmarketcap.com/headlines/news/"
        },
]
    for story in stories:
        st.write("Title:", story['title'])
        st.write("Description:", story['description'])
        st.write("Source:", story['source'])
        st.write("Published:", story['published_at'])
        st.write("URL:", story['url'])
        st.write("---------------")


# streamlit sidebar
with st.sidebar:
    st.header("OPTIONS")
    coin_list = ["BTC", "ETH", "XRP", "LTC", "BCH", "DOGE", "XLM", "AAVE", "DASH", "NEXO", "ETC",
                 "XMR", "WAVES", "MIOTA", "ICX", "DOT", "EOS", "SOL"]
    select_options = st.multiselect('Select Coins', coin_list)

    interval_options = {
        '1d': '1 Day',
        '2d': '2 Days',
        '3d': '3 Days',
        '4d': '4 Days',
        '5d': '5 Days',
        '6d': '6 Days',
        '7d': '7 Days',
        '1mo': '1 Month',
        '6mo': '6 Months',
        '1y': '1 Year'
    }
    interval = st.selectbox('Select timeframe', tuple(interval_options.keys()), format_func=lambda x: interval_options[x])

    # Number of coins to buy or sell
    num_coins = st.number_input("Number of coins to buy/sell:", min_value=0.0, step=0.01, value=0.0)

    submit_button = st.button('Submit', key='submit_button', use_container_width=True)

# Load the trained model, encoder, and scaler
scaler_ = joblib.load('myscaler.joblib')
label_encoder_ = joblib.load('mylabel_encoder.joblib')
model_ = joblib.load('mymodel.joblib')

st.header("Fred-Cryptocurrency prediction")
if submit_button:
    with st.spinner('Loading ☕...........'):

        data = execute_program(coin=select_options[0], time_string=interval)
        # replace last close Data with the predicted
        data.at[data.index[-1], 'Close'] = data.iloc[-1]["predicted"]
        starting_price = data.iloc[-1]["Open"]
        target_price = data.iloc[-1]["Close"]
        profit_price = calculate_profit_price(starting_price, target_price)
        predicted_value = target_price  # Set the predicted value to the target price
        confidence_level = calculate_confidence_level(predicted_value, target_price)
        time.sleep(5)
        st.success('Done ✅')
        st.write("The last candlestick is the predicted data.")
        st.write(f"Starting price => {round(starting_price, 2)} USD, Target price => {round(target_price, 2)} USD")
        st.write(f"Predicted value => {round(predicted_value, 2)} USD")
        st.write(f"Confidence Level: {confidence_level}")

        # Code to print the profit price
        st.write(f"Profit price => {profit_price} USD")

    # plot
    # fig, axes = mpf.plot(data[-30:], mav=100, type='candle', ylabel='Price', style='yahoo', returnfig=True)
    # Customize market colors, and place inside style you intend to use:
    mc = mpf.make_marketcolors(up='yellow', down='yellow')
    s = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc)
    # Create an all `nan` values dataframe, same size, and same index as the first:
    nans = [float('nan')] * len(data)
    cdf = pd.DataFrame(dict(Open=nans, High=nans, Low=nans, Close=nans), index=data.index)
    # Copy in the specific candles that you want to color:
    cdf.loc[str(data.tail(1).index[0])] = data.loc[str(data.tail(1).index[0])]
    # Call `mpf.plot()` twice with the two dataframes and two styles:
    fig, ax1 = mpf.plot(data[-30:], type='candle', style='yahoo', returnfig=True)
    mpf.plot(cdf[-30:], type='candle', style=s, ax=ax1[0])

    st.pyplot(fig)

    # The code below plots the trend line for actual and predicted values
    fig, ax = plt.subplots(figsize=(14, 7))
    ax.plot(data.index, data["Close"], color='blue', label='Actual')
    ax.plot(data.index, data["predicted"], color='red', label='Predicted')

    # Formatting the x-axis dates
    ax.xaxis.set_major_locator(mpl_dates.WeekdayLocator())
    ax.xaxis.set_major_formatter(mpl_dates.DateFormatter('%b %d'))

    plt.xlabel('Date')
    plt.ylabel(f'{select_options[0]} Close Price')
    plt.title(f'Actual and Predicted {select_options[0]} Close Price Trend')
    plt.legend()
    st.pyplot(fig)

    #To display top stories
    display_top_stories()
